In [3]:
import numpy as np

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [9]:
conda install bcolz

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::dask==0.19.1=py37_0
  - defaults/osx-64::anaconda==5.3.0=py37_0
  - defaults/osx-64::mkl_fft==1.0.4=py37h5d10147_1
  - defaults/osx-64::scikit-learn==0.19.2=py37h4f467ca_0
  - defaults/osx-64::bottleneck==1.2.1=py37h1d22016_1
  - defaults/osx-64::h5py==2.8.0=py37h878fce3_3
  - defaults/osx-64::statsmodels==0.9.0=py37h1d22016_0
  - defaults/osx-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/osx-64::scipy==1.1.0=py37h28f7352_1
  - defaults/osx-64::bkcharts==0.2=py37_0
  - defaults/osx-64::pywavelets==1.0.0=py37h1d22016_0
  - defaults/osx-64::patsy==0.5.0=py37_0
  - defaults/osx-64::imageio==2.4.1=py37_0
  - defaults/osx-64::mkl_random==1.0.1=py37h5d10147_1
  - defaults/osx-64::numpy-base==1.15.1=py37h8a80b8c_0
  - defaults/osx-64::numba==0.39.0=py37h6440ff4_0
  - defaults/osx-64::odo==0.5.1=py37_0
  - defaults/

In [2]:
!pip install -q fastai==0.7.0 torchtext==0.2.3

In [3]:
from fastai.conv_learner import *
from cv2 import resize
import matplotlib.gridspec as gridspec
from math import ceil
from IPython.display import HTML
from pdb import set_trace
# from scipy import ndimage

In [4]:
class SaveFeatures():
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output
    def close(self):
        self.hook.remove()

In [7]:
class FilterVisualizer():
    def __init__(self):
        #########################
        #self.model = vgg16(True).cuda().eval()
        self.model = vgg16(True).eval()
        set_trainable(self.model, False)

    def visualize(self, sz, layer, filter, upscaling_steps=12, upscaling_factor=1.2, lr=0.1, opt_steps=20, blur=None, save=False, print_losses=False):

        img = (np.random.random((sz, sz, 3)) * 20 + 128.)/255.
#         img = np.random.uniform(0, 1, size=(sz, sz, 3)).astype(np.float32)
#         median_filter_size = 4 if sz < 100 else 8
#         img = scipy.ndimage.filters.median_filter(img, [median_filter_size,median_filter_size,1])
    
        layer = children(self.model)[layer]
        activations = SaveFeatures(layer)  # register hook

        for i in range(upscaling_steps):  # scale the image up upscaling_steps times
            train_tfms, val_tfms = tfms_from_model(vgg16, sz)
            img_var = V(val_tfms(img)[None], requires_grad=True)  # convert image to Variable that requires grad
            optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
            if i > upscaling_steps/2:
                opt_steps_ = int(opt_steps*1.3)
            else:
                opt_steps_ = opt_steps
            for n in range(opt_steps_):  # optimize pixel values for opt_steps times
                optimizer.zero_grad()
                self.model(img_var)
                loss = -activations.features[0, filter].mean()
                if print_losses:
                    if i%3==0 and n%5==0:
                        print(f'{i} - {n} - {float(loss)}')
                loss.backward()
                optimizer.step()
            img = val_tfms.denorm(np.rollaxis(to_np(img_var.data),1,4))[0]
            self.output = img
            sz = int(upscaling_factor * sz)  # calculate new image size
            img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
            if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
        if save:
            self.save(layer, filter)
        activations.close()
        return np.clip(self.output, 0, 1)
    
    def most_activated(self, image, layer, limit_top=None):

        train_tfms, val_tfms = tfms_from_model(vgg16, 224)
        transformed = val_tfms(image)
#         set_trace()
        layer = children(self.model)[layer]
        activations = SaveFeatures(layer)  # register hook
        self.model(V(transformed)[None]);
        #         set_trace()
        print(activations.features.shape)
        #         set_trace()
        mean_act = [activations.features[0,i].mean().data.cpu().numpy()[0] for i in range(activations.features.shape[1])]
        activations.close()
        return mean_act
        
    def get_transformed_img(self,img,sz):
        train_tfms, val_tfms = tfms_from_model(resnet34, sz)
        return val_tfms.denorm(np.rollaxis(to_np(val_tfms(img)[None]),1,4))[0]

In [8]:
FV = FilterVisualizer()

In [9]:
FV.model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (9): ReLU(inplace)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (12): ReLU(inplace)
  (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (16): ReLU(inplace)
  (17): Conv2d(256, 256, ke

In [35]:
import requests
import base64
import pprint

In [41]:
def plot_reconstructions(imgs,layer_idx,filters,n_cols=3, cell_size=4, save_fig=False,album_hash=None):
    n_rows = ceil((len(imgs))/n_cols)

    fig,axes = plt.subplots(n_rows,n_cols, figsize=(cell_size*n_cols,cell_size*n_rows))
          
    for i,ax in enumerate(axes.flat):
        ax.grid(False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if i>=len(filters):
            pass

        ax.set_title(f'fmap {filters[i]}')

        ax.imshow(imgs[i])
    fig.suptitle(f'VGG16 Layer: {layer_idx} : {children(FV.model)[layer_idx]}', fontsize="x-large",y=1.0)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    if save_fig:
        fn = f'vgg16_layer_{layer_idx}_fmaps_{"_".join([str(f) for f in filters])}.png'
        plt.savefig(fn)
    #    link = upload_to_imgur(f'vgg16_layer_{layer_idx}_fmaps_{"_".join([str(f) for f in filters])}.png',
     #                   f'vgg16_layer_{layer_idx}_fmaps_{"_".join([str(f) for f in filters])}',album_hash)
        plt.close()
        return fn#link
    else:
        plt.show()
        return None

In [42]:
def reconstructions(layer_idx, filters,
                    init_size=56, upscaling_steps=12, 
                    upscaling_factor=1.2, 
                    opt_steps=20, blur=5,
                    lr=1e-1,print_losses=False,
                    n_cols=3, cell_size=4,
                    save_fig=False,
                    album_hash=None):
    
    if save_fig:
        assert not album_hash is None
    imgs = []
    for filter in filters:
        imgs.append(FV.visualize(init_size,layer_idx, filter, 
                                 upscaling_steps=upscaling_steps, 
                                 upscaling_factor=upscaling_factor, 
                                 opt_steps=opt_steps, blur=blur,
                                 lr=lr,print_losses=print_losses))
        
    return plot_reconstructions(imgs,layer_idx,filters,
                         n_cols=n_cols,cell_size=cell_size,
                         save_fig=save_fig,album_hash=album_hash)

In [43]:
#url = reconstructions(layer_idx=7,filters=list(range(0,3)),
fn = reconstructions(layer_idx=7,filters=list(range(0,3)),
                save_fig=True,album_hash="A5KpYOz")
print(fn)
imshow(fn)
#display(HTML(f"<img src={url} />"))




vgg16_layer_7_fmaps_0_1_2.png


'vgg16_layer_7_fmaps_0_1_2.png'

In [46]:
#url = reconstructions(layer_idx=10,filters=list(range(24,30)),
fn = reconstructions(layer_idx=10,filters=list(range(24,30)),
                save_fig=True,album_hash="A5KpYOz")

print(fn)
#display(HTML(f"<img src={url} />"))

vgg16_layer_10_fmaps_24_25_26_27_28_29.png


In [48]:
fn = reconstructions(layer_idx=40,filters=list(range(31,40)),
                save_fig=True,album_hash="A5KpYOz")

print(fn)
#display(HTML(f"<img src={url} />"))

vgg16_layer_40_fmaps_31_32_33_34_35_36_37_38_39.png
